# QA on single documents drawn from a corpus of individual files.

> Building a simple Gradio application to allow the user to ask questions about a single file. 

In [5]:
#| default_exp tools.single_document_qa

In [1]:
#| export

import os
import pandas as pd

from alhazen.core import PromptTemplateRegistry, get_langchain_llm, get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE
import alhazen.utils.jats_text_extractor as te 
from alhazen.utils.pdf_research_article_text_extractor import LAPDFBlockLoader

from databricks import sql

import gradio as gr

from InstructorEmbedding import INSTRUCTOR 

from importlib_resources import files

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import AnalyzeDocumentChain,  RetrievalQA, RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.document_loaders import UnstructuredFileLoader
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.prompts import load_prompt, PromptTemplate
from langchain.schema.vectorstore import VectorStoreRetriever
from langchain.text_splitter import NLTKTextSplitter, CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

import local_resources.xsl as xsl_sheets

from pathlib import Path

import re
import requests


[nltk_data] Downloading package punkt to /Users/gburns/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
#| export 
PROMPT_SINGLE_ARTICLE_QUESTION = {
    'name': 'SingleArticleQuestion',
    'description': 'Ask a question about a single research paper',
    'system':   
        '''You are an helpful assistant, well-versed in scientific language, but cautious about making sure what you say is accurate. 
        ''',
    'instruction': 
        '''
        Use the following pieces of context to answer the question at the end.
        If you don''t know the answer, just say that you don''t know, don''t try to make up an answer
        The question will be enclosed by double quotes. 
        The text of the progress report will be enclosed by double square brackets.
        
        CONTEXT: [[{context}]]
        
        Then, take a deep breath and think carefully before responding as precisely as possible to this question (enclosed in double quotes):
        
        QUESTION: "{question}"''',
    
    'input_variables': ["context", "question"]}


class LocalFileLangChainChatBot:
    '''Uses a simple prompt to answer questions about a collection of short PDF files.'''

    def __init__(self, 
                doc_dir, 
                llm_name,
                embed_model_name='hkunlp/instructor-large',
                section_regex=None,
                chunk_size=1000,
                chunk_overlap=100):
        
        self.doc_dir = doc_dir
        if doc_dir[-1:] != '/':
            doc_dir += '/'
        self.change_directory(self.doc_dir)
        self.prompt_registry = PromptTemplateRegistry()
        self.prompt_registry.register_new_instruction_template(PROMPT_SINGLE_ARTICLE_QUESTION)
        model_type = None

        self.section_regex = section_regex

        if llm_name == 'gpt-4':
            model_type = MODEL_TYPE.OpenAI
        elif GGUF_LOOKUP_URL.get(llm_name) is not None:
            model_type = MODEL_TYPE.LlamaCpp
        
        t = self.prompt_registry.get_instruction_template(PROMPT_SINGLE_ARTICLE_QUESTION.get('name')) 
        if model_type == MODEL_TYPE.OpenAI:
            self.prompt_template = t.generate_prompt_template()
        else:
            self.prompt_template = t.generate_llama2_prompt_template()
                
        self.llm = get_langchain_llm(llm_name)   

        self.text_splitter = RecursiveCharacterTextSplitter(
            # Set a really small chunk size, just to show.
            chunk_size = chunk_size,
            chunk_overlap  = chunk_overlap,
            length_function = len,
            is_separator_regex = False,
            add_start_index = True,
        )
        #self.text_splitter = NLTKTextSplitter(chunk_size=chunk_size, 
        #                                 chunk_overlap=chunk_overlap)

        self.embed_model = HuggingFaceInstructEmbeddings(model_name=embed_model_name,
                                                         model_kwargs={'device': 'mps'})
        self.vectorstore = Chroma("langchain_store", self.embed_model)

        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=self.vectorstore.as_retriever(),
            chain_type_kwargs={
                "prompt": self.prompt_template
            }
        )        
        
    def read_one_pdf_document(self, file_path, section_regex=None):

        pdf_loader = LAPDFBlockLoader(file_path)
        docs = pdf_loader.load()
        #html_paragraphs = '<p>'+'</p><p>'.join([d.page_content for d in docs])+'</p>'
        #html_sentences = self.sent_detector.tokenize(html_paragraphs.strip())
        # = '<br>'.join(html_sentences)

        html_text = '<p>'+re.sub('\n\n', '</p><p>', docs[0].page_content)+'</p>'
        html_text = re.sub('\n', '<br>', html_text)
        return html_text

    def read_one_nxml_document(self, file_path):

        with open(file_path, 'r') as f:
            xml = f.read()

        name = Path(file_path).stem
        doc = te.NxmlDoc(name, xml)
        df1 = doc.build_simple_document_dataframe()
        
        if self.section_regex is not None:   
            df_text = df1[df1['TAG_TREE'].str.contains(self.section_regex)]
        else: 
            df_text = df1[df1['FIG_REF'] == '']
        
        for i, row in df_text.iterrows():
            if row.TAG != 'p' and row.PLAIN_TEXT[-1:]!='.':
                row.PLAIN_TEXT += '.'
        text_list = df_text['PLAIN_TEXT'].values.tolist()

        doc_text = '\n\n'.join(text_list)

        html_text = '<p>'+re.sub('\n\n', '</p><p>', doc_text)+'</p>'
        html_text = re.sub('\n', '<br>', html_text)


        #import lxml.etree as ET
        #dom = ET.parse(file_path)
        #xsl_filename = files(xsl_sheets).joinpath('jats-to-html.xsl')
        #xslt = ET.parse(str(xsl_filename))
        #transform = ET.XSLT(xslt)
        #newdom = transform(dom)
        #html_text = ET.tostring(newdom, pretty_print=True)

        return html_text
    
    def index_document(self, doc_text, file_path):
        docs = [Document(page_content=doc_text, metadata={"file_path": file_path})]
        split_docs = self.text_splitter.split_documents(docs)
        self.vectorstore.from_documents(documents=split_docs, embedding=self.embed_model) 
        print("There are", self.vectorstore._collection.count(), "in the collection")
        self.vectorstore = Chroma("langchain_store", self.embed_model).from_documents(documents=split_docs, embedding=self.embed_model) 
        self.qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=VectorStoreRetriever(vectorstore=self.vectorstore, search_kwargs={"filter":{"file_path":file_path}}),
            chain_type_kwargs={"prompt": self.prompt_template},
        )        

    def run_batch_of_questions_over_each_file(self, questions):
        l =[]
        # List all pdf files in doc_dir by walking the whole directory tree
        pdf_file_list = []
        for root, dirs, files in os.walk(self.doc_dir):
            for file in files:
                print(file)
                if file.endswith('.pdf'):
                    loader = LAPDFBlockLoader(os.path.join(root, file))
                    docs = loader.load()
                    report_text = '\n'.join([d.page_content for d in docs])
                    for q in questions:
                        p = self.prompt_template.format(article=report_text, question=q)
                        try:
                            a = self.llm(p)
                            print('\t\t%s'%(a))
                            l.append({'file': file, 'question': q, 'answer': a})
                        except Exception as e: 
                            print('ERROR ASKING "%s" on "%s"'%(q, file)) 
                            print(e) 
        df = pd.DataFrame(l)
        return df    

    def change_directory(self, doc_dir):
        if doc_dir[-1:] != '/':
            doc_dir += '/'
        file_list = []
        dir_list = []
        with os.scandir(doc_dir) as it:
            for entry in it:
                suffix = Path(entry.path).suffix
                if entry.is_file():
                    if suffix in ['.pdf', '.PDF', '.txt', '.TXT', '.xml', '.XML', '.nxml', '.NXML']:
                        file_list.append(entry.name)
                elif entry.is_dir():
                    dir_list.append(entry.name)
        file_list.sort()
        dir_list.sort()

        # insert '..' into dir_list
        dir_list.insert(0, '..')

        self.doc_dir = doc_dir
        self.folder_df = pd.DataFrame(dir_list, columns=['folder'])
        self.file_df = pd.DataFrame(file_list, columns=['file'])
        return (doc_dir, self.folder_df, self.file_df)

    def run_gradio(self):

        def add_text(history, text):
            #print('add_text: history: %s, text: %s'%(history, text))
            history = history + [(text, None)]
            return history, gr.Textbox(value="", interactive=False)

        def select_dir(evt: gr.SelectData):
            new_dir = self.doc_dir + '/' + evt.value      
            if evt.value == '..':
                new_dir = str(Path(self.doc_dir).parent)
            print('select_dir: %s'%(new_dir))
            return self.change_directory(new_dir)

            #doc_text = f"You selected {evt.value} at {evt.index} from {evt.target}"
            #return doc_text
        
        def select_file(evt: gr.SelectData):
            file_path = self.doc_dir + '/' + evt.value  
            html_text = f"You selected {evt.value} at {evt.index} from {evt.target}"
            if file_path.endswith('.pdf'):
                html_text = self.read_one_pdf_document(file_path)
            elif file_path.endswith('.nxml') or file_path.endswith('.xml'):
                html_text = self.read_one_nxml_document(file_path)  
            
            self.index_document(html_text, file_path)
            
            return html_text, []
            
        def clear_chat(history):
            return []

        def bot(history):
            #print('bot: history: %s'%(history))
            # prompt to send to the agent is the last message from the user
            prompt = history[-1][0]
            response = self.qa_chain({"query": prompt})
            print('RESPONSE: %s'%(str(response)))
            history[-1][1] = str(response.get('result','No answer found'))
            print('WHOLE HISTORY: %s'%(history))
            return history

        with gr.Blocks() as demo:
            with gr.Tab("Corpus"):
                with gr.Row():
                    with gr.Column():
                        doc_dir = gr.Textbox(show_label=False, lines=1, value=self.doc_dir, interactive=False)
                        with gr.Row():
                            directories = gr.DataFrame(show_label=False, value=self.folder_df, interactive=False)
                            files = gr.DataFrame(show_label=False, value=self.file_df, interactive=False)
                    doc_text = gr.HTML(label="File Contents")
            
            with gr.Tab("Chat"):
                chatbot = gr.Chatbot(
                    [],
                    elem_id="chatbot",
                    bubble_full_width=False,
                    #avatar_images=(None, files(alhazen_resources).joinpath('alhazen.png'))
                )
                with gr.Row():
                    txt = gr.Textbox(
                        scale=4,
                        show_label=False,
                        placeholder="Enter text and press enter, or upload files",
                        container=False,
                    )
                    clear_btn = gr.Button("Clear")

            directories.select(select_dir, None, [doc_dir, directories, files], queue=False )   
            files.select(select_file, None, [doc_text, chatbot], queue=False )
            txt_msg = txt.submit(add_text, [chatbot, txt], [chatbot, txt], queue=False).then(bot, chatbot, chatbot)
            txt_msg.then(lambda: gr.Textbox(interactive=True), None, [txt], queue=False)                
            clear_btn.click(clear_chat, [], [chatbot], queue=False)
                
        demo.queue()
        demo.launch()


In [3]:
#os.environ['LLMS_TEMP_DIR'] = '/tmp/alhazen'
#cb = LocalFileLangChainChatBot('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/', 
#                               'llama-2-70b-chat',
#                               section_regex='[Mm]ethod')
#cb.run_gradio()
#cb.read_one_nxml_document('/Users/gburns/Documents/2023H2/cryoet_portal_rocrates/reading/cryoet_papers/de_teresa-2022-10.1101-2022.04.12.488077.nxml')

llama_model_loader: loaded meta data with 19 key-value pairs and 723 tensors from /tmp/alhazen/llama-2-70b-chat.Q5_K_M.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q5_K     [  8192, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 28672,  8192,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q5_K     [  8192, 28672,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  8192,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q5_K     [  8192,  1024,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q5_K     [  8192,  8192, 

load INSTRUCTOR_Transformer


/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:810: UserWarning: Expected 1 arguments for function <function LocalFileLangChainChatBot.run_gradio.<locals>.clear_chat at 0xf68838cc0>, received 0.
  warnings.warn(
/Users/gburns/miniconda3/envs/alhazen/lib/python3.11/site-packages/gradio/utils.py:814: UserWarning: Expected at least 1 arguments for function <function LocalFileLangChainChatBot.run_gradio.<locals>.clear_chat at 0xf68838cc0>, received 0.
  warnings.warn(


max_seq_length  512
Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


There are 0 in the collection
  The cells being vitrified in this paper are yeast cells, specifically S. pombe.RESPONSE: {'query': 'What cells are being vitrified in this paper?', 'result': '  The cells being vitrified in this paper are yeast cells, specifically S. pombe.'}
WHOLE HISTORY: [['What cells are being vitrified in this paper?', '  The cells being vitrified in this paper are yeast cells, specifically S. pombe.']]



llama_print_timings:        load time = 36521.32 ms
llama_print_timings:      sample time =    33.72 ms /    22 runs   (    1.53 ms per token,   652.49 tokens per second)
llama_print_timings: prompt eval time = 57511.37 ms /  1657 tokens (   34.71 ms per token,    28.81 tokens per second)
llama_print_timings:        eval time =  3891.46 ms /    21 runs   (  185.31 ms per token,     5.40 tokens per second)
llama_print_timings:       total time = 61550.17 ms
